# Mini project

In [2]:
def obstacle_avoidance(robot):
    left, right = robot.prox_activations(tracked_objects=["20cm"])
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return left_wheel, right_wheel

def line_following(robot):
    left, middle, right = robot.floor_sensor()
    left_wheel = 1 - right
    right_wheel = 1 - left   
    return right_wheel, left_wheel

def fear(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = left
    right_wheel = right
    return left_wheel, right_wheel

def aggression(robot):
    left, right = robot.prox_activations(tracked_objects=["ePuck"])
    left_wheel = right
    right_wheel = left
    return left_wheel, right_wheel

In [1]:
from simulator_interface import open_session, close_session
simulator, epuck1, epuck2, epuck3 = open_session(n_epucks=3)

In [2]:
close_session(simulator)

NameError: name 'simulator' is not defined

In [3]:
for e in simulator.robots:
    e.detach_all_behaviors()
    e.attach_behavior(obstacle_avoidance, freq=10)
    e.start_all_behaviors()
    
epuck1.detach_all_behaviors()
epuck1.attach_behavior(aggression, freq=10)
epuck1.attach_behavior(obstacle_avoidance, freq=10)
epuck1.start_all_behaviors()

Behavior obstacle_avoidance started
Behavior obstacle_avoidance started
Behavior obstacle_avoidance started
Behavior aggression started
Behavior obstacle_avoidance started
